In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import pandas as pd
from src.inference import fetch_next_hour_predictions, load_batch_of_features_from_store
from src.plot_utils import plot_aggregated_time_series

current_date = pd.Timestamp.now(tz="Etc/UTC")

features = load_batch_of_features_from_store(current_date)

2025-03-06 21:55:34,527 INFO: Initializing external client
2025-03-06 21:55:34,530 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 21:55:35,632 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215708
Fetching data from 2025-02-06 02:55:34.525220+00:00 to 2025-03-07 01:55:34.525220+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (39.49s) 


In [4]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-06 03:00:00
1,0,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-06 03:00:00
2,0,1,0,1,5,7,2,4,4,1,...,17,9,12,18,6,2,2,0,4,2025-03-06 03:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-06 03:00:00
4,0,0,1,2,0,4,3,2,5,4,...,5,3,1,1,1,0,1,0,7,2025-03-06 03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,259,2025-03-06 03:00:00
248,2,0,0,0,1,0,0,0,0,0,...,1,1,0,1,0,2,0,1,260,2025-03-06 03:00:00
249,0,2,2,2,7,7,16,27,17,31,...,78,57,48,34,12,0,3,1,261,2025-03-06 03:00:00
250,0,6,17,71,157,174,161,112,89,110,...,98,56,40,42,9,2,2,1,262,2025-03-06 03:00:00


In [5]:
predictions = fetch_next_hour_predictions()

2025-03-06 21:56:29,682 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 21:56:29,703 INFO: Initializing external client
2025-03-06 21:56:29,704 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 21:56:30,711 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215708
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (37.43s) 
Current UTC time: 2025-03-07 02:56:29.682420+00:00
Next hour: 2025-03-07 03:00:00+00:00
Found 252 records


In [6]:
predictions


,pickup_location_id,predicted_demand,pickup_hour
0,184,0.0,2025-03-07 03:00:00+00:00
1,82,1.0,2025-03-07 03:00:00+00:00
2,75,27.0,2025-03-07 03:00:00+00:00
3,47,0.0,2025-03-07 03:00:00+00:00
4,235,0.0,2025-03-07 03:00:00+00:00
...,...,...,...
247,260,2.0,2025-03-07 03:00:00+00:00
248,25,1.0,2025-03-07 03:00:00+00:00
249,101,0.0,2025-03-07 03:00:00+00:00
250,165,0.0,2025-03-07 03:00:00+00:00


In [7]:
top10 = (
    predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].to_list()
)

In [8]:
top10

[236, 237, 238, 132, 230, 164, 186, 100, 170, 138]

In [9]:
top10

[236, 237, 238, 132, 230, 164, 186, 100, 170, 138]

In [10]:
features["pickup_location_id"] == 22

0      False
1      False
2      False
3      False
4      False
       ...  
247    False
248    False
249    False
250    False
251    False
Name: pickup_location_id, Length: 252, dtype: bool

In [11]:
features_cp = features.copy()

In [12]:
from datetime import timedelta
features = features_cp[features_cp["pickup_location_id"] == 161]
prediction = predictions[predictions["pickup_location_id"] == 161]
time_series_columns = [
    col for col in features.columns if col.startswith("rides_t-")
]
time_series_values = [features[col].iloc[0] for col in time_series_columns] + prediction["predicted_demand"].to_list()

# Convert pickup_hour Series to single timestamp
pickup_hour = pd.Timestamp(features["pickup_hour"].iloc[0])

# Generate corresponding timestamps for the time series
time_series_dates = pd.date_range(
    start=pickup_hour - timedelta(hours=len(time_series_columns)),
    end=pickup_hour,
    freq="h",
)

# Create a DataFrame for the historical data
historical_df = pd.DataFrame(
    {"datetime": time_series_dates, "rides": time_series_values}
)

In [13]:
historical_df

,datetime,rides
0,2025-02-06 03:00:00,4.0
1,2025-02-06 04:00:00,8.0
2,2025-02-06 05:00:00,14.0
3,2025-02-06 06:00:00,40.0
4,2025-02-06 07:00:00,101.0
...,...,...
668,2025-03-05 23:00:00,195.0
669,2025-03-06 00:00:00,57.0
670,2025-03-06 01:00:00,23.0
671,2025-03-06 02:00:00,6.0
